In [18]:
# https://github.com/ageron/handson-ml/blob/master/tensorflow_graph_in_jupyter.py

from __future__ import absolute_import, division, print_function, unicode_literals

# This module defines the show_graph() function to visualize a TensorFlow graph within Jupyter.

# As far as I can tell, this code was originally written by Alex Mordvintsev at:
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb

# The original code only worked on Chrome (because of the use of <link rel="import"...>, but the version below
# uses Polyfill (copied from this StackOverflow answer: https://stackoverflow.com/a/41463991/38626)
# so that it can work on other browsers as well.

import numpy as np
import tensorflow as tf
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

## 1. save variable

- https://www.tensorflow.org/guide/saved_model
- https://www.tensorflow.org/api_docs/python/tf/train/Saver#save

In [19]:
import tensorflow as tf

tf.reset_default_graph()
# Create some variables.
v1 = tf.get_variable("v1", [3], initializer=tf.ones_initializer)
v2 = tf.get_variable("v2", [5], initializer=tf.ones_initializer)

init_op = tf.variables_initializer([v1, v2])

In [20]:
v1

<tf.Variable 'v1:0' shape=(3,) dtype=float32_ref>

In [21]:
type(v1)

tensorflow.python.ops.variables.Variable

In [22]:
v1.name, v1.op.name

('v1:0', 'v1')

By default, Saver uses the value of the ~~`tf.Variable.name`~~ `tf.Variable.op.name` property for each variable.  
1. pass them as a list.
```python
saver = tf.train.Saver([v1, v2])
```
2. Passing a list is equivalent to passing a dict with the variable **op names**
as keys:
```python
saver = tf.train.Saver({v.op.name: v for v in [v1, v2]})
```

3. However, when you create a Saver object,   
you may optionally choose names for the variables in the checkpoint files.

In [23]:
# Pass the variables as a dict:
saver = tf.train.Saver({'v1': v1, 'v2': v2})

In [24]:
# Use the saver object normally after that.
with tf.Session() as sess:
    # Initialize v1 and v2
    init_op.run()
    saver.save(sess, "tmp/model.ckpt")
    print("v1 : %s" % v1.eval())
    print("v2 : %s" % v2.eval())

v1 : [1. 1. 1.]
v2 : [1. 1. 1. 1. 1.]


## 2. restore variable

In [25]:
tf.reset_default_graph()
# Create some variables.
v1 = tf.get_variable("v1", [3], initializer=tf.zeros_initializer)
v2 = tf.get_variable("v2", [5], initializer=tf.zeros_initializer)
v3 = tf.get_variable("v3", [7], initializer=tf.zeros_initializer)

# Add ops to save and restore only `v1` and `v2` using the name "v1", "v2"
saver = tf.train.Saver({'v1': v1, 'v2': v2})

# Use the saver object normally after that.
with tf.Session() as sess:
    # If you only restore a subset of the model variables at the start of a session,
    # you have to run an initialize op for the other variables.
    # See tf.variables_initializer for more information.

    # Initialize v3 since the saver will not.
    v3.initializer.run()
    saver.restore(sess, "tmp/model.ckpt")

    print("v1 : %s" % v1.eval())
    print("v2 : %s" % v2.eval())
    print("v2 : %s" % v3.eval())


INFO:tensorflow:Restoring parameters from tmp/model.ckpt
v1 : [1. 1. 1.]
v2 : [1. 1. 1. 1. 1.]
v2 : [0. 0. 0. 0. 0. 0. 0.]


## 3. Inspect variables in a checkpoint  
We can quickly inspect variables in a checkpoint with the `inspect_checkpoint` library.

In [26]:
# import the inspect_checkpoint library
from tensorflow.python.tools import inspect_checkpoint as chkp

- print all tensors in checkpoint file

In [27]:
chkp.print_tensors_in_checkpoint_file("tmp/model.ckpt", tensor_name='', all_tensors=True)

tensor_name:  v1
[1. 1. 1.]
tensor_name:  v2
[1. 1. 1. 1. 1.]


- print only tensor v1 in checkpoint file

In [28]:
chkp.print_tensors_in_checkpoint_file("tmp/model.ckpt", tensor_name='v1', all_tensors=False)

tensor_name:  v1
[1. 1. 1.]


- print only tensor v2 in checkpoint file

In [29]:
chkp.print_tensors_in_checkpoint_file("tmp/model.ckpt", tensor_name='v2', all_tensors=False)

tensor_name:  v2
[1. 1. 1. 1. 1.]


## 4. [How to understand the term `tensor` in TensorFlow?](https://stackoverflow.com/a/37870634/8037585)

1. TensorFlow doesn't have first-class Tensor objects, meaning that there are no notion of `Tensor` in the underlying graph that's executed by the runtime.  
2. Instead the graph consists of **op nodes** connected to each other, representing operations. **An operation allocates memory for its outputs, which are available on endpoints `:0`, `:1`, etc, and you can think of each of these endpoints as a `Tensor`.**  
3. *If you have `tensor` corresponding to `nodename:0` you can fetch its value as `sess.run(tensor)` or `sess.run('nodename:0')`.*  
4. Execution granularity happens at operation level, so the `run` method will execute op which will compute all of the endpoints, not just the `:0` endpoint.  
5. It's possible to have an Op node with no outputs (like `tf.group`) in which case there are no tensors associated with it. It is not possible to have tensors without an underlying Op node.

You can examine what happens in underlying graph by doing something like this

In [30]:
tf.reset_default_graph()
value = tf.constant(1)
print(value, '\n')
print(tf.get_default_graph().as_graph_def())

Tensor("Const:0", shape=(), dtype=int32) 

node {
  name: "Const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 1
      }
    }
  }
}
versions {
  producer: 26
}



In [32]:
tf.get_default_graph().as_graph_def().node

[name: "Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
      }
      int_val: 1
    }
  }
}
]

In [14]:
with tf.Session() as sess:
    print(sess.run("Const:0"))
    print(sess.run(value))
    print(sess.run('Const')) # `Const` is op

1
1
None


So with `tf.constant` you get a single operation node, and you can fetch it using `sess.run("Const:0")` or `sess.run(value)`

Similarly, `value=tf.placeholder(tf.int32)` creates a regular node with name `Placeholder`, and you could feed it as `feed_dict={"Placeholder:0":2}` or `feed_dict={value:2}`. You can not feed and fetch a placeholder in the same `session.run` call, but you can see the result by attaching a `tf.identity` node on top and fetching that.

6. For variable

In [33]:
tf.reset_default_graph()
value = tf.Variable(tf.ones_initializer()(()))
print(value,'\n')
print(tf.get_default_graph().as_graph_def())

<tf.Variable 'Variable:0' shape=() dtype=float32_ref> 

node {
  name: "ones"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 1.0
      }
    }
  }
}
node {
  name: "Variable"
  op: "VariableV2"
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
  attr {
    key: "shared_name"
    value {
      s: ""
    }
  }
}
node {
  name: "Variable/Assign"
  op: "Assign"
  input: "Variable"
  input: "ones"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "_class"
    value {
      list {
        s: "loc:@Variable"
      }
    }
  }
  attr {
    key: "use_locking"
    value {
      b: true
    }
  }
  attr {
    key: "validate_shape"
    value {
      b: 

In [34]:
tf.get_default_graph().as_graph_def().node

[name: "ones"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 1.0
    }
  }
}
, name: "Variable"
op: "VariableV2"
attr {
  key: "container"
  value {
    s: ""
  }
}
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
    }
  }
}
attr {
  key: "shared_name"
  value {
    s: ""
  }
}
, name: "Variable/Assign"
op: "Assign"
input: "Variable"
input: "ones"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "_class"
  value {
    list {
      s: "loc:@Variable"
    }
  }
}
attr {
  key: "use_locking"
  value {
    b: true
  }
}
attr {
  key: "validate_shape"
  value {
    b: true
  }
}
, name: "Variable/read"
op: "Identity"
input: "Variable"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "_class"
  value {
    list {
      s: "loc:@Variable"
    }
  }
}
]

In [16]:
show_graph(tf.get_default_graph())

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run('Variable/read:0'))
    print(sess.run('Variable:0'))
    print(sess.run(value))
    print(sess.run('Variable')) # `Variable` is op

1.0
1.0
1.0
None


You'll see that it creates two nodes `Variable` and `Variable/read`, the `:0` endpoint is a valid value to fetch on both of these nodes.  
**However `Variable:0` has a special `ref` type meaning it can be used as an input to mutating operations.** The result of Python call `tf.Variable` is a Python `Variable` object and there's some Python magic to substitute `Variable/read:0` or `Variable:0` depending on whether mutation is necessary. Since most ops have only 1 endpoint, `:0` is dropped. Another example is `Queue` -- `close()` method will create a new `Close` op node which connects to `Queue` op. To summarize -- operations on python objects like `Variable` and `Queue` map to different underlying TensorFlow op nodes depending on usage. 

For ops like `tf.split` or `tf.nn.top_k` which create nodes with multiple endpoints, Python's `session.run` call automatically wraps output in `tuple` or `collections.namedtuple` of `Tensor` objects which can be fetched individually.

## 4. Session
```python
run(
    fetches,
    feed_dict=None,
    options=None,
    run_metadata=None
)
```
The `fetches` argument may be a single graph element, or an arbitrarily nested list, tuple, namedtuple, dict, or OrderedDict containing graph elements at its leaves. A graph element can be one of the following types:

- An `tf.Operation`. The corresponding fetched value will be None.
- A `tf.Tensor`. The corresponding fetched value will be a numpy ndarray containing the value of that tensor.
- A `tf.SparseTensor`. The corresponding fetched value will be a tf.SparseTensorValue containing the value of that sparse tensor.
- A `get_tensor_handle` op. The corresponding fetched value will be a numpy ndarray containing the handle of that tensor.
- **A `string` which is the name of a tensor or operation in the graph.**

## 5. `get_tensor_by_name` & `get_operation_by_name`  
[How to get a tensorflow op by name?
](https://stackoverflow.com/a/42686191/8037585)  
[Tensorflow: How to get a tensor by name?
](https://stackoverflow.com/a/36784246/8037585)  
[How does TensorFlow name tensors?
](https://stackoverflow.com/a/36156697/8037585)  
[Tensorflow: difference get_tensor_by_name vs get_operation_by_name?
](https://stackoverflow.com/a/48024008/8037585)  
[Tensorflow: How to get a tensor by name?
](https://stackoverflow.com/a/36613748/8037585)

In [56]:
tf.reset_default_graph()
a = tf.get_variable(name='var', shape=[], initializer=tf.zeros_initializer)
b = tf.assign_add(a, tf.convert_to_tensor(1.0), name='add')  # output Tensor
print(a, '\n', b)

out = tf.get_default_graph().get_tensor_by_name('add:0')
update = tf.get_default_graph().get_operation_by_name('add')
print(out is b)          # `out` is same with `b`, Tensor
print(update is b.op)    # `update` is same with `b.op`, Operation

with tf.Session() as sess:
    a.initializer.run()  # 0
    print(b.eval())      # 1
    
    print(update.run())  # 2, operation return None
    print(out.eval())    # 3

<tf.Variable 'var:0' shape=() dtype=float32_ref> 
 Tensor("add:0", shape=(), dtype=float32_ref)
True
True
1.0
None
3.0


Tensor names must be of the form **`<op_name>:<output_index>`**.
```python
tf.reset_default_graph()
ax = tf.constant(1.0, name='ax')
bx = ax + 2
cx = tf.identity(bx, name='cx')
print(ax)
print(bx)
print(cx)
graph = tf.get_default_graph()

# ValueError: The name 'cx' refers to an Operation, not a Tensor.
# Tensor names must be of the form "<op_name>:<output_index>".
# v1 = graph.get_tensor_by_name('cx')

v = graph.get_tensor_by_name('cx:0')
print(v)
print(v is cx)

############# output
# Tensor("ax:0", shape=(), dtype=float32)
# Tensor("add:0", shape=(), dtype=float32)
# Tensor("cx:0", shape=(), dtype=float32)
# Tensor("cx:0", shape=(), dtype=float32)
# True
```

## 6. getting variable by name  
[TensorFlow: getting variable by name
](https://stackoverflow.com/a/39027917/8037585)  
[TensorFlow: getting variable by name
](https://stackoverflow.com/a/35686754/8037585)

The `get_variable()` function creates a new variable or returns one **created** earlier by `get_variable()`. It **won't** return a variable created using `tf.Variable()`.  
Here's a quick example:

In [36]:
tf.reset_default_graph()
with tf.variable_scope("foo"):
    bar1 = tf.get_variable("bar", (2,3)) # create

with tf.variable_scope("foo", reuse=True):
    bar2 = tf.get_variable("bar")  # reuse


with tf.variable_scope("", reuse=True): # root variable scope
    bar3 = tf.get_variable("foo/bar") # reuse (equivalent to the above)

(bar1 is bar2) and (bar2 is bar3)

True

- `tf.global_variables()` and filter

In [39]:
tf.reset_default_graph()
bar1 = tf.get_variable(name='bar', initializer=1.0)

# using `var.op.name` == "variable_name"
bar2 = [var for var in tf.global_variables() if var.op.name=="bar"][0]
# `var.name` == "variable_name:0", same with the above
bar3 = [var for var in tf.global_variables() if var.name=="bar:0"][0]

(bar1 is bar2) and (bar2 is bar3)

True

- `tf.get_collection()` and scope

In [40]:
bar4 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="bar")[0]
# same 
bar5 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="bar:0")[0]

(bar1 is bar4) and (bar4 is bar5)

True

- `get_tensor_by_name()`

In [45]:
graph = tf.get_default_graph()
bar6 = graph.get_tensor_by_name('bar:0')

bar1 is bar6

False

Here `bar6` is `tf.Tensor`, but bar1 equal bar6 in value.

In [46]:
bar1, bar6

(<tf.Variable 'bar:0' shape=() dtype=float32_ref>,
 <tf.Tensor 'bar:0' shape=() dtype=float32_ref>)

## 7. `variable.name` and `variable.op.name`?  
[What's difference between `variable.name` and `variable.op.name`?
](https://stackoverflow.com/a/34729874/8037585)  

Right now the [`Variable.name`][1] property maps to **the name of the mutable [`Tensor`][2]** __in which that variable is stored__ (principally because a `Variable` can be used wherever a `Tensor` is expected). Tensor names are generated from the name of the operation that produces them (a `Variable` op in this case) and the index of the output to which that tensor corresponds.

The `op` property of a `Tensor` or a `Variable` is a pointer to the node in the TensorFlow graph that produces that tensor,

[1]: https://www.tensorflow.org/versions/master/api_docs/python/state_ops.html#Variable.name
[2]: https://www.tensorflow.org/versions/master/api_docs/python/framework.html#Tensor